# Calculate Pathweigh analysis, UDP, activity and cosistency.

In [1]:
import numpy as np
import pandas as pd
from scipy.special import gammaln, psi, factorial
from scipy.optimize import fmin_l_bfgs_b as optim
from scipy.stats import nbinom, norm, poisson, gennorm
from sklearn.mixture import GaussianMixture as GMM
from numpy import inf
from udp import *
from activity import *

infinitesimal = np.finfo(np.float).eps

## Load file

In [2]:
#Running locally, read file from path.
#data = pd.read_csv('./data/Pathweigh/GSE29013_RMA.csv', index_col=0)
#Or upload file using widget.
import pandas as pd
import ipywidgets as widgets
import io

#FileUpload limit size: https://github.com/jupyter-widgets/ipywidgets/issues/2522.
data = pd.DataFrame()

def ff(b):
    global data
    s.style.button_color = "yellow"
    for elem in b.new.values():
        print(elem['metadata']['name'])
        #with open(elem['metadata']['name'], 'wb') as file:
            #file.write(elem['content'])
        print(f'Length of data file: {len(elem["content"])}')
        data = pd.read_csv(io.BytesIO(elem['content']), index_col=0)
        print("Done")
    s.style.button_color = "lightgreen"

s = widgets.FileUpload(
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
s.style.button_color = "lightgreen"
s.observe(ff, names='value')
s

FileUpload(value={}, accept='.csv', description='Upload', style=ButtonStyle(button_color='lightgreen'))

GSE29013_RMA.csv
Length of data file: 36399849
Done


## Calculate UDP

In [3]:
#Run UDP calculation.
is_rnaseq = False #Chance to True in case of RNASeq data.
udp = calc_udp_multi_process(data, is_rnaseq)

Sat Feb  8 00:30:21 2020 Calculate UDP, is_rnaseq: False
............Sat Feb  8 00:30:40 2020 Done.


## Calculate Activity and Consistency

In [4]:
activity_obj = path_activity(udp, is_rnaseq)
activity = activity_obj.calc_activity_consistency_multi_process()

Sat Feb  8 00:31:09 2020 Init activity object
Sat Feb  8 00:31:09 2020 Calculate activity and consistency...
...Sat Feb  8 00:31:16 2020 Done.


## Select Path For Export

In [5]:
paths = pd.read_csv('pathologist.db.txt', delimiter='\t', header=None)
options = paths[0].unique()
dd = widgets.Dropdown(options=options, description='Select a path', disabled=False)
dd

Dropdown(description='Select a path', options=('granzyme a mediated apoptosis pathway(BioCarta)', 'hiv-1 nef: …

## Calculate Kegg Diagram

In [6]:
xml_result = activity_obj.xmlparser(path_id=dd.value, sample_num=0) #Select the required sample number.

Create Kegg XML for path: granzyme a mediated apoptosis pathway(BioCarta), sample: 0


## Download Kegg Diagram

In [21]:
from IPython.display import FileLink

FileLink('output_path.xml')

c:\Users\Admin\Google Drive\src\Pathweigh\output_path.xml

In [ ]:
xml_result.encode()